In [1]:
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import astropy.units as u

from collections import namedtuple

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import fits

from astropy.visualization import imshow_norm
from astropy.visualization import ZScaleInterval, LinearStretch, AsinhStretch

from astropy.nddata import Cutout2D
from astropy.convolution import Gaussian2DKernel

from astroquery.vizier import Vizier

from skimage.restoration import richardson_lucy
from skimage.exposure import rescale_intensity
from skimage.morphology import skeletonize, thin
from skimage.transform import probabilistic_hough_line

from lsst.daf.butler import Butler

In [2]:
class VisitID(namedtuple('VisitID', 'day_obs, seq_num, detector, psf, ccd_coordinates')):
    __slots__ = ()
    @property
    def visit_id(self):
        return int(f'{self.day_obs}{self.seq_num:05d}') 
    
exposure_list = [
    VisitID(20241129, 236, 7, 1.3, (3310, 260)),
    VisitID(20241129, 237, 7, 1.39, (3925, 1030)),
    VisitID(20241129, 238, 8, 1.25, (535, 2070)),
    VisitID(20241129, 239, 8, 1.37, (1300, 3000)),
    VisitID(20241129, 240, 8, 1.41, (1950, 3820)),
    VisitID(20241129, 246, 4, 1.33, (2800, 3850)),
    VisitID(20241129, 247, 7, 1.36, (3650, 690)),
    VisitID(20241129, 248, 8, 1.36, (200, 1650)),
    VisitID(20241129, 249, 8, 1.34, (940, 2570)),
    VisitID(20241129, 250, 8, 1.40, (1660, 3490)),
    VisitID(20241207, 549, 7, 2.83, (3935, 1470)),
    VisitID(20241207, 550, 8, 2.86, (410, 2350))
]

### Retrieve exposures with WCS solutions from the embargo repository

In [3]:
# Get the Exposures from the butler
RAW = 'LSSTComCam/raw/all' 
NV = 'LSSTComCam/nightlyValidation'

# Setup at USDF
butler = Butler('/repo/embargo', collections=NV)
home_dir = '/sdf/home/c/cwalter/test-fits/'
home_cleaned = '/sdf/home/c/cwalter/processed-fits/'

INFO:botocore.credentials:Found credentials in shared credentials file: /sdf/home/c/cwalter/.lsst/aws-credentials.ini


In [4]:
def get_exposure(exposure):

   calexp =  butler.get('calexp', dataId={'visit': exposure.visit_id, 'detector': exposure.detector})
   catalog =  butler.get('src', dataId={'visit': exposure.visit_id, 'detector': exposure.detector})

   wcs = WCS(calexp.getWcs().getFitsMetadata())
   info = calexp.getInfo()
   file_name = info.getMetadata()['FILENAME']

   return calexp, catalog, wcs, file_name

for i, exposure in enumerate(exposure_list):

   calexp, catalog, wcs, file_name = get_exposure(exposure)
   
   print(f'{i+1}: {exposure.day_obs} {file_name} detector={exposure.detector} at {exposure.ccd_coordinates}')
   calexp.writeFits(home_dir+file_name)

INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2024_40/conda/envs/lsst-scipipe-9.0.0-exact-ext/share/eups/Linux64/obs_base/g54c5bc6842+34edff6d8e/python/lsst/obs/base/formatters/fitsExposure.py:570: FutureWarning: Summary field [magLim] not recognized by this software version; ignoring it.
  result = self.reader.read(**self.checked_parameters)


1: 20241129 CC_O_20241129_000236_R22_S21.fits detector=7 at (3310, 260)
2: 20241129 CC_O_20241129_000237_R22_S21.fits detector=7 at (3925, 1030)
3: 20241129 CC_O_20241129_000238_R22_S22.fits detector=8 at (535, 2070)
4: 20241129 CC_O_20241129_000239_R22_S22.fits detector=8 at (1300, 3000)
5: 20241129 CC_O_20241129_000240_R22_S22.fits detector=8 at (1950, 3820)
6: 20241129 CC_O_20241129_000246_R22_S11.fits detector=4 at (2800, 3850)
7: 20241129 CC_O_20241129_000247_R22_S21.fits detector=7 at (3650, 690)
8: 20241129 CC_O_20241129_000248_R22_S22.fits detector=8 at (200, 1650)
9: 20241129 CC_O_20241129_000249_R22_S22.fits detector=8 at (940, 2570)
10: 20241129 CC_O_20241129_000250_R22_S22.fits detector=8 at (1660, 3490)
11: 20241207 CC_O_20241207_000549_R22_S21.fits detector=7 at (3935, 1470)
12: 20241207 CC_O_20241207_000550_R22_S22.fits detector=8 at (410, 2350)


In [5]:
#def deconvolve_track(exposure,)

for i, exposure in enumerate(exposure_list):

    calexp, catalog, wcs, file_name = get_exposure(exposure)
    image = calexp.image.array

    calexp_copy = calexp.clone()
    image_copy = calexp_copy.image.array

    position = exposure.ccd_coordinates
    size = (700, 550)     # Note y,x convention
    new_image = Cutout2D(image_copy, position, size, mode='trim')
    new_image.data = rescale_intensity(new_image.data, out_range=(0,1))

    # Make PSF 
    sigma = exposure.psf / 0.2  # Pixels
    stamp_size = int(20*sigma) + 1
    psf = Gaussian2DKernel(sigma, x_size=stamp_size).array
    
    iterations = int(exposure.psf*700)
    # iterations = 200
    deconvolved = richardson_lucy(new_image.data, psf, num_iter=iterations, clip=False)

    print(f'PSF is {exposure.psf}", sigma is {sigma:3.2f} pixels. {stamp_size} pixel stamp size with num_iter={iterations}.')

    image_copy[new_image.slices_original]= deconvolved
    calexp_copy.writeFits(home_cleaned+file_name.replace('CC_','CLEANED_'))

PSF is 1.3", sigma is 6.50 pixels. 131 pixel stamp size with num_iter=910.
PSF is 1.39", sigma is 6.95 pixels. 140 pixel stamp size with num_iter=972.
PSF is 1.25", sigma is 6.25 pixels. 126 pixel stamp size with num_iter=875.
PSF is 1.37", sigma is 6.85 pixels. 138 pixel stamp size with num_iter=959.
PSF is 1.41", sigma is 7.05 pixels. 141 pixel stamp size with num_iter=987.
PSF is 1.33", sigma is 6.65 pixels. 134 pixel stamp size with num_iter=931.
PSF is 1.36", sigma is 6.80 pixels. 137 pixel stamp size with num_iter=952.
PSF is 1.36", sigma is 6.80 pixels. 137 pixel stamp size with num_iter=952.
PSF is 1.34", sigma is 6.70 pixels. 135 pixel stamp size with num_iter=938.
PSF is 1.4", sigma is 7.00 pixels. 140 pixel stamp size with num_iter=979.
PSF is 2.83", sigma is 14.15 pixels. 284 pixel stamp size with num_iter=1981.
PSF is 2.86", sigma is 14.30 pixels. 287 pixel stamp size with num_iter=2002.


### Take the last catalog and overlay it with SAO catalog sources to see things are reasonable.

In [6]:
exposure = exposure_list[2]

calexp, catalog, wcs, file_name = get_exposure(exposure)
image = calexp.image.array

position = exposure.ccd_coordinates
size = (700, 550)     # Note y,x convention
new_image = Cutout2D(image, position, size, mode='trim')

# Make PSF 
sigma = exposure.psf / 0.2  # Pixels
stamp_size = int(20*sigma) + 1
print(f'PSF is {exposure.psf}, sigma is {sigma} pixels. PSF image is {stamp_size} pixels')

psf = Gaussian2DKernel(sigma, x_size=stamp_size).array

new_image.data = rescale_intensity(new_image.data, out_range=(0,1))
#deconvolved = richardson_lucy(new_image.data, psf, num_iter=2000, clip=False)
deconvolved = richardson_lucy(new_image.data, psf, num_iter=int(exposure.psf*700), clip=False)


/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2024_40/conda/envs/lsst-scipipe-9.0.0-exact-ext/share/eups/Linux64/obs_base/g54c5bc6842+34edff6d8e/python/lsst/obs/base/formatters/fitsExposure.py:570: FutureWarning: Summary field [magLim] not recognized by this software version; ignoring it.
  result = self.reader.read(**self.checked_parameters)


PSF is 1.25, sigma is 6.25 pixels. PSF image is 126 pixels


In [7]:
# Take the sources and put them into a table including sky coordinates from WCS
pixel_x = catalog.getColumnView().getX()
pixel_y = catalog.getColumnView().getY()
object_fluxes = catalog['base_CircularApertureFlux_3_0_instFlux']
sky_coordinates = wcs.pixel_to_world(pixel_x, pixel_y)

image_object_table = Table([pixel_x, pixel_y, sky_coordinates.ra, sky_coordinates.dec, object_fluxes], 
names=('x','y','_RAJ2000', '_DEJ2000', 'flux'))

image_object_table.sort('flux', reverse = True)
selected_sources = image_object_table
selected_sources[:10]

x,y,_RAJ2000,_DEJ2000,flux
,,deg,deg,
float64,float64,float64,float64,float64
3541.8431285137053,2650.438083622778,35.38337401220092,4.737062070702756,2404940.0
3541.8406642126356,2650.438737699134,35.383373954609084,4.737061940915319,2404068.0
1571.3098604023482,194.01195364591447,35.207416214189216,4.736956281334058,1548539.75
1571.3098604023482,194.01195364591447,35.207416214189216,4.736956281334058,1548539.75
1049.8917550088318,2341.724064566623,35.28286481951238,4.639594276228904,1239480.625
1260.3861077648075,3374.80978742866,35.33527888996819,4.612782811386216,1093696.375
1260.3861084025043,3374.8097883260757,35.335278890029606,4.6127828113826705,1093696.375
2153.0,436.0,35.23826511222149,4.753798455845787,1016648.3125


In [8]:
# Match those sources to the a catalogue
matched_stars = Vizier(catalog = 'I/345/gaia2', timeout=240).query_region(selected_sources, 
                                                             radius="30s", inner_radius="1s")[0]
matched_stars

_q,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Source,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,Dup,FG,e_FG,Gmag,e_Gmag,FBP,e_FBP,BPmag,e_BPmag,FRP,e_FRP,RPmag,e_RPmag,BP-RP,RV,e_RV,Teff,AG,E_BP-RP_,Rad,Lum
,deg,mas,deg,mas,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,mag,,,mag,mag,,,mag,mag,mag,km / s,km / s,K,mag,mag,solRad,solLum
int32,float64,float64,float64,float64,int64,float64,float32,float64,float32,float64,float32,uint8,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float32,float64,float32,float32,float32,float64
1,35.38357480254,0.0483,4.73787615595,0.0455,2516617537128063872,1.3477,0.0582,2.950,0.106,-3.052,0.094,0,7678,5.363,15.9753,0.0008,3469,47.31,16.5008,0.0148,6669,76.39,15.2017,0.0124,1.2990,--,--,4871.00,0.2550,0.1190,0.61,0.190
2,35.38357480254,0.0483,4.73787615595,0.0455,2516617537128063872,1.3477,0.0582,2.950,0.106,-3.052,0.094,0,7678,5.363,15.9753,0.0008,3469,47.31,16.5008,0.0148,6669,76.39,15.2017,0.0124,1.2990,--,--,4871.00,0.2550,0.1190,0.61,0.190
3,35.20409423429,0.1964,4.73256254674,0.2306,2516525895410286208,1.0997,0.2807,32.091,0.444,-14.010,0.406,0,654.9,1.873,18.6479,0.0031,168.5,6.782,19.7848,0.0437,734.4,5.94,17.5971,0.0088,2.1877,--,--,--,--,--,--,--
4,35.20409423429,0.1964,4.73256254674,0.2306,2516525895410286208,1.0997,0.2807,32.091,0.444,-14.010,0.406,0,654.9,1.873,18.6479,0.0031,168.5,6.782,19.7848,0.0437,734.4,5.94,17.5971,0.0088,2.1877,--,--,--,--,--,--,--
5,35.27799225227,2.4340,4.64200506632,3.3633,2516521463004023936,--,--,--,--,--,--,0,158.4,3.354,20.1892,0.0230,119.3,6.395,20.1598,0.0582,323.4,14.02,18.4875,0.0470,1.6723,--,--,--,--,--,--,--
10,35.38169686261,0.7910,4.68832167114,0.9458,2516615574327462912,0.2700,0.9196,4.153,1.986,-0.500,2.835,0,122.9,1.148,20.4642,0.0101,82.18,7.058,20.5644,0.0932,77.76,10.15,20.0351,0.1418,0.5294,--,--,--,--,--,--,--
11,35.38169686261,0.7910,4.68832167114,0.9458,2516615574327462912,0.2700,0.9196,4.153,1.986,-0.500,2.835,0,122.9,1.148,20.4642,0.0101,82.18,7.058,20.5644,0.0932,77.76,10.15,20.0351,0.1418,0.5294,--,--,--,--,--,--,--
14,35.14748573143,0.0985,4.67057945051,0.1024,2516525139496035968,-0.0748,0.1266,-0.200,0.221,-1.724,0.196,0,1846,2.625,17.5227,0.0015,900,9.873,17.9658,0.0119,1416,11.22,16.8841,0.0086,1.0817,--,--,--,--,--,--,--


In [ ]:
wcs_axis = plt.subplot(projection=wcs)

star_ra, star_dec = matched_stars['RA_ICRS'], matched_stars['DE_ICRS']
star_coords = SkyCoord(star_ra, star_dec, unit=(u.deg, u.deg),frame='icrs')

# Display Image
imshow_norm(image, interval=ZScaleInterval(n_samples=700), interpolation='none', stretch=LinearStretch())
#wcs_axis.imshow(image, interpolation='none', vmin=-70, vmax=150)

wcs_axis.autoscale(False)
wcs_axis.scatter_coord(star_coords, s=10, linewidth=0.2, edgecolor='white', facecolor='none')

plt.grid(color='white', ls='dotted')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')

In [ ]:
plt.imshow(image, interpolation='none', origin='lower', vmin=-70, vmax=150)

#display = afwDisplay.Display()
#display.scale('asinh', 'zscale') 
#display.mtv(calexp.image)

In [ ]:
plt.imshow(new_image.data, interpolation='none', origin='lower', vmin=new_image.data.min(), vmax=new_image.data.max())
plt.colorbar()

In [ ]:
plt.imshow(deconvolved, interpolation='none', origin='lower', vmin=deconvolved.min(), vmax=20)#deconvolved.max())
plt.colorbar()

In [ ]:
plt.imshow(image_copy, interpolation='none', origin='lower', vmin=0, vmax=1)
#new_image.plot_on_original(color='white') 
plt.colorbar()

In [ ]:
xdim, ydim = new_image.data.shape

#chris = new_image.data[int(xdim*.86):int(xdim*.93), int(ydim*.83):int(ydim*.95)]
#cropped_image = new_image.data[600:650, 460:520]
#cropped_deconvolved = deconvolved[600:650, 460:520]

cropped_image = new_image.data[int(xdim*.86):int(xdim*.93), int(ydim*.83):int(ydim*.95)]
cropped_deconvolved = deconvolved[int(xdim*.86):int(xdim*.93), int(ydim*.83):int(ydim*.95)]

plt.imshow(cropped_image, interpolation='none', origin='lower')
plt.colorbar()

In [ ]:
plt.imshow(cropped_deconvolved, interpolation='none', origin='lower', vmin=0, vmax=1)
plt.colorbar()

In [ ]:
fig, axes = plt.subplot_mosaic('AB', sharey=True, constrained_layout=True)

axes['A'].imshow(cropped_image, interpolation='none', origin='lower', vmin=0, vmax=.8)
axes['B'].imshow(cropped_deconvolved, interpolation='none', origin='lower', vmin=0, vmax=4)


In [ ]:
#skeleton = skeletonize(cropped_deconvolved>.001)
skeleton = thin(cropped_deconvolved>.01)
lines = probabilistic_hough_line(skeleton, threshold=10, line_length=15, line_gap=2)

#plt.imshow(cropped_deconvolved, origin='lower', vmax=1)
plt.imshow(skeleton, origin='lower', interpolation='none', vmax=.01)

print(f'found {len(lines)} lines')
for line in lines:
    p0, p1 = line
    plt.plot((p0[0], p1[0]), (p0[1], p1[1]))

plt.colorbar()